In [51]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import load_model


feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

#load dataset
train_data = pd.read_csv('/content/drive/MyDrive/ML/NSL-KDD/KDDTrain+.txt',names=feature)
test_data = pd.read_csv('/content/drive/MyDrive/ML/NSL-KDD/KDDTest+.txt',names=feature)

train_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [52]:
#drop difficulty column
train_data.drop('difficulty',axis=1,inplace=True)

In [53]:
#change label to DoS, Probe, U2R, R2L
train_data['label'] = train_data['label'].replace(['back','land','neptune','pod','smurf','teardrop','apache2','udpstorm','processtable','worm','mailbomb'],'DoS')
train_data['label'] = train_data['label'].replace(['ipsweep','nmap','portsweep','satan','mscan','saint'],'Probe')
train_data['label'] = train_data['label'].replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L')
train_data['label'] = train_data['label'].replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R')

train_data['label'].value_counts()

normal    67343
DoS       45927
Probe     11656
R2L         995
U2R          52
Name: label, dtype: int64

In [54]:
# use standard scaler to scale the data
scaler = StandardScaler()
train_data.iloc[:,4:41] = scaler.fit_transform(train_data.iloc[:,4:41])
train_data

<ipython-input-54-8706c4c05625>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train_data.iloc[:,4:41] = scaler.fit_transform(train_data.iloc[:,4:41])


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.818890,-0.782367,-0.280282,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,normal
1,0,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.035688,-1.161030,2.736852,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal
2,0,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.809857,-0.938287,-0.174417,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,DoS
3,0,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,1.258754,1.066401,-0.439078,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,normal
4,0,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,1.258754,1.066401,-0.439078,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.818890,-0.938287,-0.121485,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,DoS
125969,8,udp,private,SF,-0.007744,-0.004883,-0.014089,-0.089486,-0.007736,-0.095076,...,1.159389,0.977304,-0.386146,-0.447834,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal
125970,0,tcp,smtp,SF,-0.007382,-0.004823,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.773724,-0.893738,-0.121485,-0.480197,-0.289103,0.979238,-0.624871,-0.355014,-0.376387,normal
125971,0,tcp,klogin,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.972455,-1.094207,-0.174417,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,DoS


In [55]:
# label encoding: DoS=0, Probe=1, R2L=2, U2R=3, Normal=4
train_data['label'] = train_data['label'].replace(['DoS','Probe','R2L','U2R', 'normal'],[0,1,2,3,4])

# one hot encoding
train_data = pd.get_dummies(train_data,columns=['protocol_type','service','flag'])

train_data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0,0,0,0,0,0,0,0,1,0
1,0,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0,0,0,0,0,0,0,0,1,0
2,0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0,0,0,0,1,0,0,0,0,0
3,0,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,0,0,0,0,0,0,0,0,1,0
4,0,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0,0,0,0,1,0,0,0,0,0
125969,8,-0.007744,-0.004883,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0,0,0,0,0,0,0,0,1,0
125970,0,-0.007382,-0.004823,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,0,0,0,0,0,0,0,0,1,0
125971,0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0,0,0,0,1,0,0,0,0,0


In [56]:
# split data into train and test
X = train_data.drop('label',axis=1)
y = train_data['label']

# one hot encoding for y
y = pd.get_dummies(y).values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# reshape data to 3D array
X_train = np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)
X_train.shape

(100778, 122, 1)

In [57]:
X_test = np.array(X_test).reshape(X_test.shape[0],X_test.shape[1],1)
X_test.shape

(25195, 122, 1)

In [58]:
# build model
model = Sequential()
model.add(Conv1D(32,2,activation='relu',input_shape=(X_train.shape[1],1),padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(64,2,activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(128,2,activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [59]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_17 (Conv1D)          (None, 122, 32)           96        
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 61, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 61, 64)            4160      
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 30, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_19 (Conv1D)          (None, 30, 128)           16512     
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 15, 128)         

In [60]:
# train model
model.fit(X_train,y_train,epochs=50,batch_size=3000,validation_data=(X_test,y_test))

Epoch 1/50
34/34 [==============================] - 4s 48ms/step - loss: 0.8083 - accuracy: 0.8628 - val_loss: 0.1898 - val_accuracy: 0.9524
Epoch 2/50
34/34 [==============================] - 1s 38ms/step - loss: 0.1677 - accuracy: 0.9573 - val_loss: 0.1194 - val_accuracy: 0.9618
Epoch 3/50
34/34 [==============================] - 1s 38ms/step - loss: 0.1075 - accuracy: 0.9681 - val_loss: 0.0789 - val_accuracy: 0.9750
Epoch 4/50
34/34 [==============================] - 1s 37ms/step - loss: 0.0731 - accuracy: 0.9760 - val_loss: 0.0551 - val_accuracy: 0.9817
Epoch 5/50
34/34 [==============================] - 1s 37ms/step - loss: 0.0502 - accuracy: 0.9838 - val_loss: 0.0448 - val_accuracy: 0.9863
Epoch 6/50
34/34 [==============================] - 1s 37ms/step - loss: 0.0394 - accuracy: 0.9878 - val_loss: 0.0398 - val_accuracy: 0.9884
Epoch 7/50
34/34 [==============================] - 1s 38ms/step - loss: 0.0356 - accuracy: 0.9890 - val_loss: 0.0362 - val_accuracy: 0.9893
Epoch 8/50
34

In [61]:
# save model
model.save('/content/drive/MyDrive/ML/NSL-KDD/deepids.h5')

In [62]:
# load model
model = load_model('/content/drive/MyDrive/ML/NSL-KDD/deepids.h5')